In [1]:
import tensorflow as tf
import numpy as np 
from datasets import load_dataset
ds = load_dataset("ylecun/mnist")

2026-02-11 17:54:27.640306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770832467.803642      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770832467.851266      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770832468.235895      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770832468.235935      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770832468.235937      17 computation_placer.cc:177] computation placer alr

README.md: 0.00B [00:00, ?B/s]

mnist/train-00000-of-00001.parquet:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

mnist/test-00000-of-00001.parquet:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [2]:
training_images = np.array(ds['train']['image'])
training_labels = np.array(ds['train']['label'])

test_images = np.array(ds['test']['image'])
test_labels = np.array(ds['test']['label'])

In [3]:
training_images = training_images / 255.0

In [4]:
def create_and_compile_model():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(28,28)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256,activation=tf.nn.relu),
        tf.keras.layers.Dense(10,activation = tf.nn.softmax)
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=['accuracy']
    )
    return model

In [5]:
untrained_model = create_and_compile_model()

2026-02-11 17:55:00.671685: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
class EarlyStoppingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        if logs['accuracy'] >= 0.98:
            self.model.stop_training = True
            print("| Reached 98% accuracy so cancelling training")

In [7]:
def train_mnist(training_images,training_labels):
    model = create_and_compile_model()
    history = model.fit(training_images,training_labels,epochs=10,callbacks=[EarlyStoppingCallback()])
   
    return history

In [8]:
training_history = train_mnist(training_images,training_labels)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8913 - loss: 0.3771
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9717 - loss: 0.0975
Epoch 3/10
1865/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9818 - loss: 0.0612| Reached 98% accuracy so cancelling training
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9818 - loss: 0.0612
